In [ ]:
# default_exp util

# Loss functions

> API details.


In [ ]:
#export 
import numpy as np
import torch as th
import math as m
import numba.cuda as cuda

@cuda.jit
def prox_D_gaussian_kernel(z, z_hat, a, beta, a_strides):
    """

    :param z:           D x K x My x Mx x 2
    :param z_hat:       D x K x My x Mx x 2
    :param a:           D x K x My x Mx
    :param beta:        1
    :param a_strides:   (4,)
    :return:
    """
    n = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    D, K, MY, MX, _ = z.shape
    N1 = D * K * MY * MX

    d = n // a_strides[0]
    k = (n - d * a_strides[0]) // a_strides[1]
    my = (n - d * a_strides[0] - k * a_strides[1]) // a_strides[2]
    mx = (n - d * a_strides[0] - k * a_strides[1] - my * a_strides[2]) // a_strides[3]

    if n < N1:
        z_hatc = z_hat[d, k, my, mx, 0] + 1j * z_hat[d, k, my, mx, 1]
        abs_zhat_c = abs(z_hatc)
        if abs_zhat_c != 0:
            sgn_zhat = z_hatc / abs_zhat_c
            fac = (a[d, k, my, mx] + beta * abs_zhat_c) / (1.0 + beta)
            zc = fac * sgn_zhat
            z[d, k, my, mx, 0] = zc.real
            z[d, k, my, mx, 1] = zc.imag

In [ ]:
#export 
def prox_D_gaussian(z, z_hat, a, beta):
    """
    Proximal operator of the Gaussian log-likelihood.

    :param z:           D x K x My x Mx x 2, updated exit waves
    :param z_hat:       D x K x My x Mx x 2, current model exit waves
    :param a:           D x K x My x Mx,     measured amplitudes
    :param beta:        float                hyperparameter

    :return: z
    """
    gpu = cuda.get_current_device()
    stream = th.cuda.current_stream().cuda_stream
    threadsperblock = gpu.MAX_THREADS_PER_BLOCK // 2
    blockspergrid = m.ceil(np.prod(np.array(a.shape)) / threadsperblock)
    strides = th.tensor(a.stride()).to(z.device)
    prox_D_gaussian_kernel[blockspergrid, threadsperblock, stream](z, z_hat, a, beta, strides)
    return z

In [ ]:
#export 
@cuda.jit
def gradz_poisson_sparse_kernel(out, z, a_indices, a_counts, no_count_indicator, total_cts):
    """

    :param z:           D x K x My x Mx x 2
    :param z_hat:       D x K x My x Mx x 2
    :param a_indices:   D x K x counts
    :param a_counts:    D x K x counts
    :param beta:        float
    :param frame_dimensions: (2,)
    :param no_count_indicator: float or int
    :return:
    """

    d, k = cuda.grid(2)
    D, K, MY, MX, _ = z.shape
    if d < D and k < K:
        if total_cts[d,k] > 0:
            for i in range(a_indices[d, k].shape[0]):
                idx1d = a_indices[d, k, i]
                if idx1d != no_count_indicator:
                    my = idx1d // MX
                    mx = idx1d - my * MX
                    zc = z[d, k, my, mx, 0] + 1j * z[d, k, my, mx, 1]
                    abs_zc = abs(zc)
                    # if abs_zc != 0:
                    fac = 1 - (a_counts[d, k, i] / (abs_zc**2+1e-2))
                    # else:
                    #     fac = 1 - (a_counts[d, k, i] / 1e-3)
                    zc *= fac
                    out[d, k, my, mx, 0] = zc.real
                    out[d, k, my, mx, 1] = zc.imag

In [ ]:
#export 
def gradz_poisson_sparse(out, z, a_indices, a_counts):
    """
    Proximal operator of the Gaussian log-likelihood. Sparse version

    :param z:           D x K x My x Mx x 2, updated exit waves
    :param z_hat:       D x K x My x Mx x 2, current model exit waves
    :param a_indices:   D x K x cts,     measured amplitude indices
    :param a_counts:    D x K x cts,     measured amplitude counts
    :param beta:        float                hyperparameter

    :return: z
    """
    gpu = cuda.get_current_device()
    stream = th.cuda.current_stream().cuda_stream
    threadsperblock = (2, 32)
    blockspergrid = tuple(np.ceil(np.array(z.shape[:2]) / threadsperblock).astype(np.int32))
    no_count_indicator = th.iinfo(a_indices.dtype).max
    total_cts = th.sum(a_counts,2)
    gradz_poisson_sparse_kernel[blockspergrid, threadsperblock, stream](out, z, a_indices, a_counts, no_count_indicator,
                                                                        total_cts)
    return z

In [ ]:
#export 
@cuda.jit
def gradz_gaussian_sparse_kernel(out, z, a_indices, a_counts, no_count_indicator):
    """

    :param z:           D x K x My x Mx x 2
    :param z_hat:       D x K x My x Mx x 2
    :param a_indices:   D x K x counts
    :param a_counts:    D x K x counts
    :param beta:        float
    :param frame_dimensions: (2,)
    :param no_count_indicator: float or int
    :return:
    """
    d, k = cuda.grid(2)
    D, K, MY, MX, _ = z.shape
    if d < D and k < K:
        for i in range(a_indices[d, k].shape[0]):
            idx1d = a_indices[d, k, i]
            if idx1d != no_count_indicator:
                my = idx1d // MX
                mx = idx1d - my * MX
                zc = z[d, k, my, mx, 0] + 1j * z[d, k, my, mx, 1]
                abs_zc = abs(zc)
                if abs_zc != 0:
                    fac = 1 - (float(a_counts[d, k, i]) / abs_zc)
                else:
                    fac = 1 - (float(a_counts[d, k, i]) / 1e-3)
                zc *= fac
                out[d, k, my, mx, 0] = zc.real
                out[d, k, my, mx, 1] = zc.imag

In [ ]:
#export 

def gradz_gaussian_sparse(out, z, a_indices, a_counts):
    """
    Proximal operator of the Gaussian log-likelihood. Sparse version

    :param z:           D x K x My x Mx x 2, updated exit waves
    :param z_hat:       D x K x My x Mx x 2, current model exit waves
    :param a_indices:   D x K x cts,     measured amplitude indices
    :param a_counts:    D x K x cts,     measured amplitude counts
    :param beta:        float                hyperparameter

    :return: z
    """
    gpu = cuda.get_current_device()
    stream = th.cuda.current_stream().cuda_stream
    threadsperblock = (2, 32)
    blockspergrid = tuple(np.ceil(np.array(z.shape[:2]) / threadsperblock).astype(np.int32))
    no_count_indicator = th.iinfo(a_indices.dtype).max
    gradz_gaussian_sparse_kernel[blockspergrid, threadsperblock, stream](out, z, a_indices, a_counts, no_count_indicator)
    return z

In [ ]:
#export 
import torch as th
import numpy as np

from numba import cuda


@cuda.jit
def sparse_amplitude_loss_kernel(a_model, indices_target, counts_target, loss, grad, frame_dimensions,
                                 no_count_indicator):
    k = cuda.grid(1)
    K, _ = indices_target.shape
    MY, MX = frame_dimensions
    if k < K:
        for i in range(indices_target[k].shape[0]):
            idx1d = indices_target[k, i]
            my = idx1d // MX
            mx = idx1d - my * MX
            if idx1d != no_count_indicator:
                grad[k, my, mx] = 1 - (counts_target[k, i] / (a_model[k, my, mx] + 1e-2))
                cuda.atomic.add(loss, (0), (a_model[k, my, mx] - counts_target[k, i]) ** 2)
                
def sparse_amplitude_loss(a_model, indices_target, counts_target, frame_dimensions):
    """

    :param a_model:             K x M1 x M2
    :param indices_target:      K x num_max_counts
    :param counts_target:       K x num_max_counts
    :param frame_dimensions:    2
    :return: loss (1,), grad (K x M1 x M2)
    """
    threadsperblock = (256,)
    blockspergrid = tuple(np.ceil(np.array(indices_target.shape[0]) / threadsperblock).astype(np.int32))

    loss = th.zeros((1,), device=a_model.device, dtype=th.float32)
    grad = th.ones_like(a_model)
    no_count_indicator = th.iinfo(indices_target.dtype).max
    sparse_amplitude_loss_kernel[blockspergrid, threadsperblock](a_model.detach(), indices_target.detach(),
                                                                 counts_target.detach(), loss.detach(), grad.detach(),
                                                                 frame_dimensions, no_count_indicator)
    return loss, grad

import math as m
import cmath as cm
@cuda.jit
def sparse_smooth_truncated_amplitude_loss_kernel(a_model, indices_target, counts_target, loss, grad, no_count_indicator, eps):
    n = cuda.grid(1)
    K, MY, MX = a_model.shape
    N = K * MY * MX

    k = n // (MY * MX)
    my = (n - k * (MY * MX)) // MX
    mx = (n - k * (MY * MX) - my * MX)
    
    if n < N:
        idx1d = mx + my * MX
        a_measure = 0
        a_model = a_model[k, my, mx]
        for i in range(indices_target.shape[1]):
            if indices_target[k, i] == idx1d and no_count_indicator != indices_target[k, i]:
                a_measure = m.sqrt(counts_target[k, i])
            
        if a_model < eps * a_measure:
            grad[k, my, mx] =  (1-1/eps) 
            loss_k = (1-eps)/2*(a_measure**2-(1/eps)*a_model**2)
        else:
            grad[k, my, mx] =  1 - (a_measure/a_model)
            loss_k = .5 * abs(a_model - a_measure)**2
        loss[k] = loss_k
        # cuda.atomic.add(loss, (k), loss_k)
            

def sparse_smooth_truncated_amplitude_loss(a_model, indices_target, counts_target, frame_dimensions, eps=0.1):
    """
    Smooth truncated amplitude loss from Chang et al., Overlapping Domain Decomposition Methods for Ptychographic Imaging, (2020)
    
    :param a_model:             K x M1 x M2
    :param indices_target:      K x num_max_counts
    :param counts_target:       K x num_max_counts
    :param frame_dimensions:    2
    :return: loss (K,), grad (K x M1 x M2)
    """
    
    threadsperblock = (256,)
    blockspergrid = tuple(np.ceil(np.array(np.prod(a_model.shape)) / threadsperblock).astype(np.int32))

    loss = th.zeros((a_model.shape[0],), device=a_model.device, dtype=th.float32)
    grad = th.ones_like(a_model)
    no_count_indicator = th.iinfo(indices_target.dtype).max
    
    sparse_smooth_truncated_amplitude_loss_kernel[blockspergrid, threadsperblock](a_model.detach(), indices_target.detach(),
                                                                 counts_target.detach(), loss.detach(), grad.detach(),
                                                                 no_count_indicator, eps)
    return loss, grad

@cuda.jit
def sparse_smooth_truncated_amplitude_prox_kernel(a_model, indices_target, counts_target, loss, grad, no_count_indicator, eps, lam):
    n = cuda.grid(1)
    K, MY, MX = a_model.shape
    N = K * MY * MX

    k = n // (MY * MX)
    my = (n - k * (MY * MX)) // MX
    mx = (n - k * (MY * MX) - my * MX)
    
    if n < N:
        idx1d = mx + my * MX
        a_measure = 0
        a_model = a_model[k, my, mx]
        for i in range(indices_target.shape[1]):
            if indices_target[k, i] == idx1d and no_count_indicator != indices_target[k, i]:
                a_measure = m.sqrt(counts_target[k, i])
            
        # if a_model < eps * a_measure:
        #     grad[k, my, mx] =  max(0, lam * a_measure/(lam-((1-eps)/eps)))/ a_model
        #     loss_k = (1-eps)/2*(a_measure**2-(1/eps)*a_model**2)
        # else:
        #     grad[k, my, mx] =  ((a_measure+lam * a_model)/(1+lam))/ a_model
        #     loss_k = .5 * abs(a_model - a_measure)**2
            
        grad[k, my, mx] =  a_measure/ a_model
        loss_k = .5 * abs(a_model - a_measure)**2    
        
        loss[k] = loss_k
            

def sparse_smooth_truncated_amplitude_prox(a_model, indices_target, counts_target, frame_dimensions, eps=0.5, lam=6e-1):
    """
    Smooth truncated amplitude loss from Chang et al., Overlapping Domain Decomposition Methods for Ptychographic Imaging, (2020)
    
    :param a_model:             K x M1 x M2
    :param indices_target:      K x num_max_counts
    :param counts_target:       K x num_max_counts
    :param frame_dimensions:    2
    :return: loss (K,), grad (K x M1 x M2)
    """
    
    threadsperblock = (256,)
    blockspergrid = tuple(np.ceil(np.array(np.prod(a_model.shape)) / threadsperblock).astype(np.int32))

    loss = th.zeros((a_model.shape[0],), device=a_model.device, dtype=th.float32)
    grad = th.ones_like(a_model)
    no_count_indicator = th.iinfo(indices_target.dtype).max
    
    sparse_smooth_truncated_amplitude_prox_kernel[blockspergrid, threadsperblock](a_model.detach(), indices_target.detach(),
                                                                 counts_target.detach(), loss.detach(), grad.detach(),
                                                                 no_count_indicator, eps, lam)
    return loss, grad

@cuda.jit
def sparse_amplitude_prox_kernel(a_model, indices_target, counts_target, loss, grad, no_count_indicator, eps, lam):
    n = cuda.grid(1)
    K, MY, MX = a_model.shape
    N = K * MY * MX

    k = n // (MY * MX)
    my = (n - k * (MY * MX)) // MX
    mx = (n - k * (MY * MX) - my * MX)
    
    if n < N:
        idx1d = mx + my * MX
        a_measure = 0
        a_model = a_model[k, my, mx]
        for i in range(indices_target.shape[1]):
            if indices_target[k, i] == idx1d and no_count_indicator != indices_target[k, i]:
                a_measure = m.sqrt(counts_target[k, i])
            
        grad[k, my, mx] =  a_measure/ a_model
        loss_k = .5 * abs(a_model - a_measure)**2    
        
        loss[k] = loss_k
            

def sparse_amplitude_prox(a_model, indices_target, counts_target, frame_dimensions, eps=0.5, lam=6e-1):
    """
    Smooth truncated amplitude loss from Chang et al., Overlapping Domain Decomposition Methods for Ptychographic Imaging, (2020)
    
    :param a_model:             K x M1 x M2
    :param indices_target:      K x num_max_counts
    :param counts_target:       K x num_max_counts
    :param frame_dimensions:    2
    :return: loss (K,), grad (K x M1 x M2)
    """
    
    threadsperblock = (256,)
    blockspergrid = tuple(np.ceil(np.array(np.prod(a_model.shape)) / threadsperblock).astype(np.int32))

    loss = th.zeros((a_model.shape[0],), device=a_model.device, dtype=th.float32)
    grad = th.ones_like(a_model)
    no_count_indicator = th.iinfo(indices_target.dtype).max
    
    sparse_amplitude_prox_kernel[blockspergrid, threadsperblock](a_model.detach(), indices_target.detach(),
                                                                 counts_target.detach(), loss.detach(), grad.detach(),
                                                                 no_count_indicator, eps, lam)
    return loss, grad